In [1]:
import xarray as xr
import numpy as np
import os
from glob import glob
from tqdm import tqdm
import pandas as pd
import math
from multiprocess import Pool

## my own directory
os.chdir("/g/data/k10/dl6968/Semi-variogram_AU/")

In [2]:
## daily
df = pd.read_csv("./data/BoM_daily_stations.csv")
## mannually remove some faulty stations
df = df[(df['ID'] != 40592) & (df['ID'] != 40593) & (df['ID'] != 58090) & (df["End_Year"]>=1960)]
daily_lat = []
daily_lon = []
for i in range(0, len(df)):
    daily_lat.append(df["Latitude"].iloc[i])
    daily_lon.append(df["Longitude"].iloc[i])

In [3]:
## for stations to be a neighbour station
df_neighbour = df[df["Years"] >=10]
## for stations to be a spec station
df_spec = df[df["Years"] >=20]

In [4]:
percentile = 0.90

## make a dataset for percentile first

In [5]:
stn_bom_p90 = {}
faulty_stn = []
for stn_id in tqdm(df_neighbour["ID"],leave=True, position=0):
    bom_id = str(stn_id).zfill(6)
    try:
        ds = xr.open_dataset(f"/g/data/w40/dl6968/BoM_daily_stations/netcdf/{bom_id}.nc")
    except:
        faulty_stn.append(stn_id)
        continue
        
    precip_da = ds["prcp"].sel(time=slice("1940-03-02", "2024-06-30"))
    filtered_da = precip_da.where(precip_da > 1, drop=True)
    if len(filtered_da)==0:
        stn_bom_p90[str(stn_id)] = -1
    else:
        p90 = filtered_da.quantile(percentile, dim='time')
        stn_bom_p90[str(stn_id)] = p90.values
    ds.close()

100%|██████████| 11800/11800 [13:05<00:00, 15.02it/s]


In [6]:
p90_list = []
stn_list = []
for keys in stn_bom_p90.keys():
    stn_list.append(int(keys))
    p90_list.append(float(stn_bom_p90[keys]))

In [7]:
df_p90 = pd.DataFrame.from_dict({"ID": stn_list,"P90":p90_list})

In [8]:
df_p90.to_csv("./data/BoM_stn_p90.csv")